## Chatting With SEC Knowledge Graph

First Run `Insert_Into_DB_from_Json.ipynb` Notebook before you proceed in this notebook.

In [1]:
import os
import textwrap

# import necessarylangchain libraries
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains import GraphCypherQAChain
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# graph 
from yfiles_jupyter_graphs import GraphWidget
from IPython.display import display, Markdown

from dotenv import load_dotenv

In [2]:
def display_markdown(mark_text: str):
    '''
    Function to display the markdown text in notebook
    '''
    return display(Markdown(mark_text))

In [3]:
# load the env variables from .env file
load_dotenv()

# Global constants
VECTOR_INDEX_NAME = "form_10k_chunks"
VECTOR_NODE_LABEL = "Chunk"
VECTOR_SOURCE_PROPERTY = "text"
VECTPR_EMBEDDING_PROPERTIES = ["textEmedding"]

In [4]:
graph = Neo4jGraph(enhanced_schema=True)

### Create the index for full text Search

In [5]:
graph.query("""
CREATE FULLTEXT INDEX fullTextManagerNames
    IF NOT EXISTS
    FOR (mgr:Manager)
    ON EACH [mgr.managerName]
""")


graph.query("""
CREATE FULLTEXT INDEX fullTextCompanyNames
    IF NOT EXISTS
    FOR (com:Company)
    ON EACH [com.companyName]
""")


[]

### Explore the Updated SEC Documents Graph

**Enhanced Schema Information**

Using the enhanced schema version allows the system to automatically scan databases for example values and calculate distribution metrics. If a node property has fewer than 10 distinct values, all possible values are returned in the schema. Otherwise, only a single example value per node and relationship property is provided.

In [6]:
# update the schema 
graph.refresh_schema()
display_markdown(graph.schema)

Node properties:
- **Chunk**
  - `chunkSeqId`: INTEGER Min: 0, Max: 19
  - `chunkId`: STRING Example: "0001650372-23-000040-item1-chunk0000"
  - `cusip6`: STRING Available options: ['G06242', '31428X', '65249B', '36241U', '003783', '64110D', '697435', '958102', '654106', 'G7945J']
  - `id`: INTEGER Min: 0, Max: 543
  - `text`: STRING Example: ">ITEM 1. BUSINESS Company Overview   Our mission i"
  - `names`: LIST Min Size: 1, Max Size: 4
  - `cik`: STRING Available options: ['1650372', '1048911', '1564708', '1126741', '1490054', '1002047', '1327567', '106040', '320187', '1137789']
  - `source`: STRING Available options: ['https://www.sec.gov/Archives/edgar/data/1650372/00', 'https://www.sec.gov/Archives/edgar/data/1048911/00', 'https://www.sec.gov/Archives/edgar/data/1564708/00', 'https://www.sec.gov/Archives/edgar/data/1126741/00', 'https://www.sec.gov/Archives/edgar/data/1490054/00', 'https://www.sec.gov/Archives/edgar/data/1002047/00', 'https://www.sec.gov/Archives/edgar/data/1327567/00', 'https://www.sec.gov/Archives/edgar/data/106040/000', 'https://www.sec.gov/Archives/edgar/data/320187/000', 'https://www.sec.gov/Archives/edgar/data/1137789/00']
  - `formId`: STRING Available options: ['0001650372-23-000040', '0000950170-23-033201', '0001564708-23-000368', '0001558370-23-011516', '0001096906-23-001489', '0000950170-23-027948', '0001327567-23-000024', '0000106040-23-000024', '0000320187-23-000039', '0001137789-23-000049']
  - `f10kItem`: STRING Available options: ['item1', 'item1a', 'item7', 'item7a']
- **Form**
  - `cusip6`: STRING Available options: ['G06242', '31428X', '65249B', '36241U', '003783', '64110D', '697435', '958102', '654106', 'G7945J']
  - `source`: STRING Available options: ['https://www.sec.gov/Archives/edgar/data/1650372/00', 'https://www.sec.gov/Archives/edgar/data/1048911/00', 'https://www.sec.gov/Archives/edgar/data/1564708/00', 'https://www.sec.gov/Archives/edgar/data/1126741/00', 'https://www.sec.gov/Archives/edgar/data/1490054/00', 'https://www.sec.gov/Archives/edgar/data/1002047/00', 'https://www.sec.gov/Archives/edgar/data/1327567/00', 'https://www.sec.gov/Archives/edgar/data/106040/000', 'https://www.sec.gov/Archives/edgar/data/320187/000', 'https://www.sec.gov/Archives/edgar/data/1137789/00']
  - `id`: INTEGER Min: 544, Max: 553
  - `formId`: STRING Available options: ['0001650372-23-000040', '0000950170-23-033201', '0001564708-23-000368', '0001558370-23-011516', '0001096906-23-001489', '0000950170-23-027948', '0001327567-23-000024', '0000106040-23-000024', '0000320187-23-000039', '0001137789-23-000049']
  - `names`: LIST Min Size: 1, Max Size: 4
- **Company**
  - `cusip6`: STRING Available options: ['G06242', '31428X', '65249B', '36241U', '003783', '64110D', '697435', '958102', '654106', 'G7945J']
  - `location`: LIST Min Size: 2, Max Size: 2
  - `companyName`: STRING Available options: ['ATLASSIAN CORP PLC', 'FEDEX CORP', 'NEWS CORP NEW', 'GSI TECHNOLOGY INC', 'APPLE INC', 'NETAPP INC', 'PALO ALTO NETWORKS INC', 'WESTERN DIGITAL CORP.', 'NIKE INC', 'SEAGATE TECHNOLOGY']
  - `names`: LIST Min Size: 1, Max Size: 4
  - `id`: INTEGER Min: 554, Max: 563
  - `cusip`: STRING Available options: ['G06242104', '31428X106', '65249B109', '36241U106', '003783310', '64110D104', '697435105', '958102105', '654106103', 'G7945J104']
  - `companyAddress`: STRING Available options: ['431 El Camino Real, Santa Clara, CA 95050, USA', 'Delaware, USA', '1211 6th Ave, New York, NY 10036, USA', '1213 Elko Dr, Sunnyvale, CA 94089, USA', 'Cupertino, CA, USA', 'Headquarters Dr, San Jose, CA 95134, USA', '3000 Tannery Way, Santa Clara, CA 95054, USA', 'San Jose, 5601 Great Oaks Pkwy, United States', 'Oregon, USA', '2445 Augustine Dr, Santa Clara, CA 95054, USA']
- **Manager**
  - `managerName`: STRING Example: "KINGDON CAPITAL MANAGEMENT, L.L.C."
  - `managerCik`: STRING Example: "1000097"
  - `managerAddress`: STRING Example: "152 WEST 57TH STREET, 50TH FLOOR, NEW YORK, NY, 10"
  - `id`: INTEGER Min: 564, Max: 3301
  - `location`: LIST Min Size: 2, Max Size: 2
- **Address**
  - `country`: STRING Example: "United States"
  - `id`: INTEGER Min: 3302, Max: 4165
  - `location`: LIST Min Size: 2, Max Size: 2
  - `city`: STRING Example: "New York"
  - `state`: STRING Example: "New York"
Relationship properties:
- **SECTION**
  - `f10kItem: STRING` Available options: ['item1', 'item1a', 'item7', 'item7a']
- **OWNS_STOCK_IN**
  - `value: FLOAT` Min: 0.0, Max:  1.1899154484E13
  - `shares: INTEGER` Min: 0, Max:  1922059328
  - `reportCalendarOrQuarter: STRING` Available options: ['2023-06-30', '2023-03-31', '2021-09-30', '2020-12-31', '2022-06-30', '2022-09-30']
The relationships:
(:Chunk)-[:NEXT]->(:Chunk)
(:Chunk)-[:PART_OF]->(:Form)
(:Form)-[:SECTION]->(:Chunk)
(:Company)-[:LOCATED_AT]->(:Address)
(:Company)-[:FILED]->(:Form)
(:Manager)-[:LOCATED_AT]->(:Address)
(:Manager)-[:OWNS_STOCK_IN]->(:Company)

### Visualize the Graph database

In [7]:
from neo4j import GraphDatabase

# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

### Full text search

In [8]:
# full text search a manager named Royal Bank
graph.query("""
CALL db.index.fulltext.queryNodes(
    "fullTextManagerNames", 
    "Royal Bank"
) YIELD node, score
RETURN node.managerName, score
LIMIT 1

""")

[{'node.managerName': 'Royal Bank of Canada', 'score': 4.431276321411133}]

### Do Some query in Knowledge Graph

In [9]:
# check the address of a random Manager
graph.query("""
MATCH (mgr:Manager)-[LOCATED_AT]->(addr:Address)
RETURN mgr, addr
limit 1
""")

[{'mgr': {'managerCik': '1424381',
   'managerAddress': '650 Madison Avenue, 25th Floor, New York, NY, 10022',
   'location': [-73.9713457, 40.7639879],
   'id': 1145,
   'managerName': 'LAKEWOOD CAPITAL MANAGEMENT, LP'},
  'addr': {'country': 'United States',
   'city': 'New York',
   'location': [-73.9821429, 40.7584882],
   'id': 3302,
   'state': 'New York'}}]

In [10]:
# find the loaction of Royal Bank
graph.query("""
CALL db.index.fulltext.queryNodes(
         "fullTextManagerNames", 
         "royal bank"
  ) YIELD node, score
WITH node as mgr LIMIT 1
MATCH (mgr:Manager)-[:LOCATED_AT]->(addr:Address)
RETURN mgr.managerName, addr
""")

[{'mgr.managerName': 'Royal Bank of Canada',
  'addr': {'country': 'Canada',
   'city': 'Toronto',
   'location': [-79.3805647, 43.6508267],
   'id': 3303,
   'state': 'Ontario'}}]

In [11]:
# determine which state has the most investment firms
graph.query("""
MATCH p=(:Manager)-[:LOCATED_AT]->(address:Address)
RETURN address.state as state, count (address.state) as numManagers
ORDER BY numManagers DESC
LIMIT 10
""")

[{'state': 'New York', 'numManagers': 304},
 {'state': 'California', 'numManagers': 302},
 {'state': 'Massachusetts', 'numManagers': 146},
 {'state': 'Pennsylvania', 'numManagers': 138},
 {'state': 'Texas', 'numManagers': 125},
 {'state': 'Illinois', 'numManagers': 121},
 {'state': 'Florida', 'numManagers': 115},
 {'state': 'Connecticut', 'numManagers': 77},
 {'state': 'Ohio', 'numManagers': 76},
 {'state': 'New Jersey', 'numManagers': 69}]

In [12]:
# determine whihc state has the most companies
graph.query("""
MATCH p=(:Company)-[:LOCATED_AT]->(address:Address)
RETURN address.state as state, count(address.state) as numCompanies
ORDER BY numCompanies DESC
LIMIT 10
""")

[{'state': 'California', 'numCompanies': 7},
 {'state': 'Delaware', 'numCompanies': 1},
 {'state': 'New York', 'numCompanies': 1},
 {'state': 'Oregon', 'numCompanies': 1}]

In [13]:
# which city in california with most investment firms
graph.query("""
MATCH p=(:Company)-[:LOCATED_AT]->(address:Address)
    WHERE address.state='California'
RETURN address.city as citty, count(address.city) as numCompanies
ORDER BY numCompanies DESC
""")

[{'citty': 'Santa Clara', 'numCompanies': 3},
 {'citty': 'San Jose', 'numCompanies': 2},
 {'citty': 'Sunnyvale', 'numCompanies': 1},
 {'citty': 'Cupertino', 'numCompanies': 1}]

In [14]:
# which city in California has the most companies listed
graph.query("""
MATCH p=(:Company)-[:LOCATED_AT]->(address:Address)
    WHERE address.state = 'California'
RETURN address.city as city, count(address.city) as numCompanies
    ORDER BY numCompanies DESC
""")

[{'city': 'Santa Clara', 'numCompanies': 3},
 {'city': 'San Jose', 'numCompanies': 2},
 {'city': 'Sunnyvale', 'numCompanies': 1},
 {'city': 'Cupertino', 'numCompanies': 1}]

In [15]:
# what are the top investment firms in San Francisco?
graph.query("""
MATCH (mgr:Manager)-[:LOCATED_AT]->(address:Address),
      (mgr)-[owns:OWNS_STOCK_IN]->(:Company)
WHERE address.city = "San Francisco"
RETURN mgr.managerName, sum(owns.value) as totalInvestmentValue
ORDER BY totalInvestmentValue DESC
LIMIT 10
""")

[{'mgr.managerName': 'Dodge & Cox', 'totalInvestmentValue': 3889236092000.0},
 {'mgr.managerName': 'WELLS FARGO & COMPANY/MN',
  'totalInvestmentValue': 2177580039000.0},
 {'mgr.managerName': 'CHARLES SCHWAB INVESTMENT MANAGEMENT INC',
  'totalInvestmentValue': 1944847519000.0},
 {'mgr.managerName': 'Parallax Volatility Advisers, L.P.',
  'totalInvestmentValue': 694023723000.0},
 {'mgr.managerName': 'PARNASSUS INVESTMENTS, LLC',
  'totalInvestmentValue': 211068925000.0},
 {'mgr.managerName': 'Spyglass Capital Management LLC',
  'totalInvestmentValue': 98135259000.0},
 {'mgr.managerName': 'Valiant Capital Management, L.P.',
  'totalInvestmentValue': 52124040000.0},
 {'mgr.managerName': 'Ensemble Capital Management, LLC',
  'totalInvestmentValue': 42355370000.0},
 {'mgr.managerName': 'Woodline Partners LP',
  'totalInvestmentValue': 41497384000.0},
 {'mgr.managerName': 'Alta Park Capital, LP',
  'totalInvestmentValue': 38959909000.0}]

In [16]:
## what companies are located in Santa Clara
graph.query("""
MATCH (com:Company)-[:LOCATED_AT]->(address:Address)
    WHERE address.city = "Santa Clara"
RETURN com.company
""")

[{'com.company': None}, {'com.company': None}, {'com.company': None}]

In [17]:
graph.query("""
MATCH (address:Address)
SET address.location = point({longitude: address.longitude, latitude: address.latitude})
""")

[]

In [18]:
# what companies are near Santa Clara
graph.query("""
  MATCH (sc:Address)
    WHERE sc.city = "Santa Clara"
  MATCH (com:Company)-[:LOCATED_AT]->(comAddr:Address)
    WHERE point.distance(sc.location, comAddr.location) < 1000000
  RETURN com.companyName, com.companyAddress
""")

[]

In [19]:
##which investment  firms are near Santa Clara
graph.query("""
MATCH (address:Address)
    WHERE address.city = "Santa Clara"
WITH address
MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, managerAddress.location) < 10000
RETURN mgr.managerName, managerAddress
""")

[]

### Writing Cypher with LLM

The `GraphCypherQAChain` includes a default Cypher generation template. However, if our graph database requires more complex Cypher queries, we can create a custom Cypher query generation template to meet our specific needs.

In [20]:
CYPHER_GENERATION_TEMPLATE = """Task: Generate a Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema. Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that ask for anything other than constructing a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# What investment firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName

The question is:
{question}"""


In [21]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)


# initiate the google gemini pro model
LLM = GoogleGenerativeAI(model="models/gemini-1.5-pro-latest", temperature=0.0)

In [22]:
cypherChain = GraphCypherQAChain.from_llm(
    LLM,
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [23]:
def prettyCypherChain(question: str) -> str:
    '''
    Display output in a way that looks nice. 
    `invoke` method returns the dict
    '''
    response = cypherChain.invoke(question)
    print(textwrap.fill(response['result'], 60))

In [24]:
cypherChain.invoke("What investment firms are in San Francisco?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Manager)-[:LOCATED_AT]->(a:Address)
WHERE a.city = 'San Francisco'
RETURN m.managerName

Full Context:
[{'m.managerName': 'PARNASSUS INVESTMENTS, LLC'}, {'m.managerName': 'SKBA CAPITAL MANAGEMENT LLC'}, {'m.managerName': 'ROSENBLUM SILVERMAN SUTTON S F INC /CA'}, {'m.managerName': 'CHARLES SCHWAB INVESTMENT MANAGEMENT INC'}, {'m.managerName': 'WELLS FARGO & COMPANY/MN'}, {'m.managerName': 'Dodge & Cox'}, {'m.managerName': 'Strait & Sound Wealth Management LLC'}, {'m.managerName': 'Sonoma Private Wealth LLC'}, {'m.managerName': 'Fund Management at Engine No. 1 LLC'}, {'m.managerName': 'SELDON CAPITAL LP'}]

> Finished chain.


{'query': 'What investment firms are in San Francisco?',
 'result': "I don't know the answer. \n"}

In [25]:
prettyCypherChain("What investment firms are in Menlo Park?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Manager)-[:LOCATED_AT]->(a:Address)
WHERE a.city = 'Menlo Park'
RETURN m.managerName

Full Context:
[{'m.managerName': 'Bordeaux Wealth Advisors LLC'}, {'m.managerName': 'Opes Wealth Management LLC'}, {'m.managerName': 'Solstein Capital, LLC'}, {'m.managerName': 'Stamos Capital Partners, L.P.'}, {'m.managerName': 'TIEMANN INVESTMENT ADVISORS, LLC'}, {'m.managerName': 'SCGE MANAGEMENT, L.P.'}, {'m.managerName': 'Nelson Capital Management, LLC'}, {'m.managerName': 'Jasper Ridge Partners, L.P.'}, {'m.managerName': 'CROSSLINK CAPITAL INC'}]

> Finished chain.
I don't know the answer.


In [26]:
prettyCypherChain("What companies are in Santa Clara?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (c:Company)-[:LOCATED_AT]->(a:Address)
WHERE a.city = 'Santa Clara'
RETURN c.companyName

Full Context:
[{'c.companyName': 'ATLASSIAN CORP PLC'}, {'c.companyName': 'PALO ALTO NETWORKS INC'}, {'c.companyName': 'SEAGATE TECHNOLOGY'}]

> Finished chain.
ATLASSIAN CORP PLC, PALO ALTO NETWORKS INC, and SEAGATE
TECHNOLOGY are in Santa Clara.


In [27]:
prettyCypherChain("What investment firms are near Santa Clara?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Manager)-[:LOCATED_AT]->(a:Address)
WHERE a.city = 'Santa Clara'
RETURN m.managerName

Full Context:
[]

> Finished chain.
I don't know the answer.


### Expand the prompt to teach the LLM new Cypher patterns

In [28]:
CYPHER_GENERATION_TEMPLATE = """Task: Generate a Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that ask for anything other than constructing a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# What investment firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName

# What investment firms are near Santa Clara?
MATCH (address:Address)
WHERE address.city = "Santa Clara"
WITH address
MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
WHERE point.distance(address.location, managerAddress.location) < 10000
RETURN mgr.managerName, managerAddress

The question is:
{question}"""


In [29]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

cypherChain = GraphCypherQAChain.from_llm(
    LLM,
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [30]:
prettyCypherChain("What investment firms are near Santa Clara?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (address:Address)
WHERE address.city = "Santa Clara"
WITH address
MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
WHERE point.distance(address.location, managerAddress.location) < 10000
RETURN mgr.managerName

Full Context:
[]

> Finished chain.
I don't know the answer.


### Expand the query to retrieve information from the Form 10K chunks

In [31]:
CYPHER_GENERATION_TEMPLATE = """Task: Generate a Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that ask for anything other than constructing a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# What investment firms are in San Francisco?
MATCH (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
WHERE mgrAddress.city = 'San Francisco'
RETURN mgr.managerName

# What investment firms are near Santa Clara?
MATCH (address:Address)
WHERE address.city = "Santa Clara"
WITH address
MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
WHERE point.distance(address.location, managerAddress.location) < 10000
RETURN mgr.managerName, managerAddress

# What does Palo Alto Networks do?
CALL db.index.fulltext.queryNodes(
    "fullTextCompanyNames", 
    "Palo Alto Networks"
) YIELD node, score
WITH node AS com
MATCH (com)-[:FILED]->(f:Form),
      (f)-[s:SECTION]->(c:Chunk)
WHERE s.f10kItem = "item1"
RETURN c.text

The question is:
{question}"""


In [32]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

cypherChain = GraphCypherQAChain.from_llm(
    LLM,
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)


In [33]:
prettyCypherChain("What does Palo Alto Networks do?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
CALL db.index.fulltext.queryNodes("fullTextCompanyNames", "Palo Alto Networks") YIELD node, score WITH node AS com MATCH (com)-[:FILED]->(f:Form)-[s:SECTION]->(c:Chunk) WHERE s.f10kItem = "item1" RETURN c.text

Full Context:
[{'c.text': '>Item 1. Business\nGeneral\nPalo Alto Networks, Inc. is a global cybersecurity provider with a vision of a world where each day is safer and more secure than the one before. We were incorporated in 2005 and are headquartered in Santa Clara, California.\nWe empower enterprises, organizations, service providers, and government entities to protect themselves against today’s most sophisticated cyber threats. Our cybersecurity platforms and services help secure enterprise users, networks, clouds, and endpoints by delivering comprehensive cybersecurity backed by industry-leading artificial intelligence and automation. We are a leading provider of zero trust solutions, starting with next-generatio